# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [83]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target,columns = ['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [86]:
print(len(x_train), len(x_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [87]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

In [88]:
result = linreg.fit(x_train, y_train)

In [89]:
y_hat_train = result.predict(x_train)
y_hat_test = result.predict(x_test)

Calculate the residuals and the mean squared error

In [90]:
from sklearn.metrics import mean_squared_error as mse

In [91]:
train_MSE = mse(y_train, y_hat_train)
test_MSE = mse(y_test, y_hat_test)

In [92]:
train_MSE

23.38421469608518

In [93]:
test_MSE

15.421295141899307

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [94]:
def kfolds(data, k):
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers
    data = pd.DataFrame(data)
    fold_size = len(data)//k
    remainder = len(data)%k
    folds = []
    start = 0
    for i in range(0,len(data), fold_size):
        if len(data) - i < fold_size:
            count = 0
            for j in range(start,len(data)):
                folds[count].append(data.iloc[start+count,:])
                count += 1
        else:
            folds.append(data[start:start + fold_size])
            start += fold_size
            
    return folds

### Apply it to the Boston Housing Data

In [95]:
# Make sure to concatenate the data again
len(bos_data)

506

In [96]:
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [97]:
folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [100]:
test_errs = []
train_errs = []
k=12

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat(fold for i, fold in enumerate(folds) if i != n)
    test = folds[n]
    # Fit a linear regression model
    model = linreg.fit(train.iloc[:,:-1], train.iloc[:,-1])
    #Evaluate Train and Test Errors
    y_hat_train = model.predict(train.iloc[:,:-1])
    y_hat_test = model.predict(test.iloc[:,:-1])
    train_mse = mse(train.iloc[:,-1], y_hat_train)
    test_mse = mse(test.iloc[:,-1], y_hat_test)
    train_errs.append(train_mse)
    test_errs.append(test_mse)
    print(f'train error is {train_mse}, test error is {test_mse}')
print(np.mean(train_errs), np.mean(test_errs))
# print(train_errs)
# print(test_errs)

train error is 23.663952225292256, test error is 13.758664308561501
train error is 22.673539795064432, test error is 17.01753808747737
train error is 19.231459518262113, test error is 37.586711433976525
train error is 14.988360350292503, test error is 57.4906914151425
train error is 22.345830313906692, test error is 24.73827740382132


IndexError: list index out of range

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!